Guillermo Blanco Núñez <br>
Pablo Díaz Blanco<br>
ap-2526-p3-ap-11-01


# Práctica 3: RNNs

## Objetivo

Los objetivos de la práctica son:
1. Diseñar y entrenar una red neuronal residual lineal para un problema de clasificación binaria.
2. Implementar un método de fine-tuning eficiente que permita adaptar el modelo a una variante del
problema sin volver a entrenar todos los parámetros.
3. Analizar las ventajas y limitaciones del método

## Preparación del dataset


Importa todas las librerías necesarias.

In [2]:
import os
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, Model, callbacks

Función para cargar archivos .pkl (pickle).

In [3]:
def load_pkl(path):
    with open(path, "rb") as f:
        return pickle.load(f)

Importa el split del dataset Higgs, descargado del aula virtual.

In [4]:
DATA_PATH = "../Higgs_DS/"

train_file = os.path.join(DATA_PATH, "higgs_train.pkl")
test_file  = os.path.join(DATA_PATH, "higgs_test.pkl")
extra_file = os.path.join(DATA_PATH, "higgs_extra.pkl")



train_data = load_pkl(train_file)
test_data  = load_pkl(test_file)
extra_data = load_pkl(extra_file)


Comprueba que el conjunto de datos esté correctamente importado al mostrar por pantalla esta información del mismo.

In [5]:
print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Extra shape:", extra_data.shape)

Train shape: (2000000, 29)
Test shape: (500000, 29)
Extra shape: (2000000, 29)


Convertir a arrays de numpy y separar los datos de las etiquetas

In [6]:
train_np = np.asarray(train_data)
test_np  = np.asarray(test_data)

X = train_np[:, :-1].astype(np.float32)
y = train_np[:, -1].astype(np.float32)

X_test = test_np[:, :-1].astype(np.float32)
y_test = test_np[:, -1].astype(np.float32)


Creación de train y validation

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

Calculo de la media y la desviación típica y aplicar normalización 

In [8]:
mean_train = X_train.mean(axis=0)
std_train  = X_train.std(axis=0)

std_train_safe = np.where(std_train == 0, 1.0, std_train)

X_train_norm = (X_train - mean_train) / std_train_safe
X_val_norm   = (X_val   - mean_train) / std_train_safe
X_test_norm  = (X_test  - mean_train) / std_train_safe


Comprobación:

In [9]:
print("Train:", X_train_norm.shape, y_train.shape)
print("Val:  ", X_val_norm.shape,   y_val.shape)
print("Test: ", X_test_norm.shape,  y_test.shape)

print("\nMedia primeras 5 features del TRAIN normalizado:")
print(X_train_norm.mean(axis=0)[:3])

print("\nSTD primeras 5 features del TRAIN normalizado:")
print(X_train_norm.std(axis=0)[:3])


Train: (1600000, 28) (1600000,)
Val:   (400000, 28) (400000,)
Test:  (500000, 28) (500000,)

Media primeras 5 features del TRAIN normalizado:
[ 1.1567707e-06 -2.7320066e-05  4.5669823e-09]

STD primeras 5 features del TRAIN normalizado:
[0.9983659 0.9999674 1.0000007]


## Definición de los hiperparámetros
- Constantes: 
    1. EPOCHS: número máximo de épocas de entrenamiento para cada configuración.
    2. BATCH_SIZE: número de ejemplos que se procesan juntos en cada actualización de pesos.
    3. INPUT_SIZE: número de características de entrada (columnas) del dataset Higgs
    4. USE_EARLY_STOPPING: indica si se usa parada temprana para evitar sobreentrenar.
    5. PATIENCE: número de épocas sin mejora permitido antes de activar el early stopping.

- Cada tupla de la lista hyperparams representa un conjunto de hiperparámetros con la forma:
    
    (used_dim, n_blocks, opt_name, dropout, use_batchnorm)
- Explicación de cada uno:
    1. used_dim: tamaño del espacio interno, es decir, número de neuronas en cada capa densa de los bloques residuales. 
    2. n_blocks: número de bloques residuales encadenados. Cada bloque incluye dos capas lineales y una conexión residual.
    3. opt_name: tipo de optimizador utilizado.
    4. dropout: porcentaje de neuronas que se apagan aleatoriamente tras la activación (solo cuando no se usa BatchNorm).
    5. use_batchnorm: booleano que indica si se aplica Batch Normalization en las capas densas. 

In [ ]:
EPOCHS = 50
BATCH_SIZE = 512
INPUT_SIZE = X_train_norm.shape[1]
PATIENCE = 20

hyperparams = [
    (64,  2, "Adam",    0.0, False),
    (64,  2, "Adam",    0.2, False),
    (128, 2, "Adam",    0.0, False),
    (128, 2, "Adam",    0.3, False),

    (64,  2, "Adam",    0.0, True),
    (128, 3, "Adam",    0.0, True),

    (64,  2, "RMSprop", 0.0, False),
    (64,  3, "SGD",     0.0, False),
]


## Construcción de una red residual
- build_residual_MLP: construye y compila una red residual densa con bloques residuales para el problema de clasificación binaria del dataset Higgs. Recibe como entrada el tamaño del vector de características y los hiperparámetros del modelo. Primero crea una capa de entrada y otra densa inicial. Luego construye n_blocks capas residuales, donde cada una tiene dos capas lineales. El resultado de esto se suma a la entrada original aplicando ReLU al final. Por último, añade una capa Dense con activación sigmoide para clasifiación binaria. Selecciona el optimizador y crea el callback de early stopping. Los parámetros son:
    1. input_size: número de características de entrada.
    2. used_dim: número de neuronas en las capas densas internas de la red
    3. n_blocks: número de bloques residuales encadenados. 
    4. opt_name: nombre del optimizador a utilizar para el entrenamiento.
    5. dropout: tasa de dropout aplicada tras la activación cuando no se usa Batch Normalization.
    6. use_batchnorm: booleano que indica si se aplica Batch Normalization en las capas densas.
    7. use_early_stopping: indica si se debe crear y devolver un callback de parada temprana (EarlyStopping) junto con el modelo.
    8. patience: número de épocas sin mejora en la métrica de validación permitidas antes de activar el early stopping.

In [ ]:

def build_residual_MLP(input_size, used_dim, n_blocks, opt_name, dropout, use_batchnorm):

    if use_batchnorm and dropout > 0.0:
        print("[AVISO] use_batchnorm=True y dropout>0. Ignorando dropout para no mezclar BN y Dropout.")
        dropout = 0.0

    inputs = layers.Input(shape=(input_size,))

    x = layers.Dense(used_dim)(inputs)

    if use_batchnorm:
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    else:
        x = layers.ReLU()(x)
        if dropout > 0.0:
            x = layers.Dropout(dropout)(x)

    for i in range(n_blocks):
        residual = x  

        z = layers.Dense(used_dim)(x)
        if use_batchnorm:
            z = layers.BatchNormalization()(z)
            z = layers.ReLU()(z)
        else:
            z = layers.ReLU()(z)
            if dropout > 0.0:
                z = layers.Dropout(dropout)(z)

        z = layers.Dense(used_dim)(z)
        if use_batchnorm:
            z = layers.BatchNormalization()(z)

        x = layers.Add()([residual, z])
        x = layers.ReLU()(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = Model(inputs, outputs)

    if opt_name == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
    elif opt_name == "SGD":
        opt = tf.keras.optimizers.SGD(learning_rate=0.03, momentum=0.9)
    elif opt_name == "RMSprop":
        opt = tf.keras.optimizers.RMSprop(learning_rate=1e-3, rho=0.9)
    else:
        raise ValueError(f"Optimiser '{opt_name}' not supported")

    model.compile(optimizer=opt)

    
    return model


In [ ]:
def make_datasets(X_train, y_train, X_val, y_val, batch_size):
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_ds = train_ds.shuffle(buffer_size=len(X_train)).batch(batch_size)

    val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    val_ds = val_ds.batch(batch_size)

    return train_ds, val_ds


def train_one_model(
    model,
    X_train, y_train,
    X_val, y_val,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    patience=PATIENCE
):
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    train_acc_metric = tf.keras.metrics.BinaryAccuracy()
    val_acc_metric = tf.keras.metrics.BinaryAccuracy()

    train_ds, val_ds = make_datasets(X_train, y_train, X_val, y_val, batch_size)

    history = []
    best_val_acc = 0.0
    best_weights = model.get_weights()
    best_epoch = 0
    epochs_without_improve = 0

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        train_loss_sum = 0.0
        train_steps = 0

        for step, (x_batch, y_batch) in enumerate(train_ds):
            with tf.GradientTape() as tape:
                logits = model(x_batch, training=True)
                loss_value = loss_fn(y_batch, logits)
                if model.losses:
                    loss_value += tf.add_n(model.losses)

            grads = tape.gradient(loss_value, model.trainable_weights)
            model.optimizer.apply_gradients(zip(grads, model.trainable_weights))

            train_acc_metric.update_state(y_batch, logits)

            train_loss_sum += float(loss_value.numpy())
            train_steps += 1

        train_loss = train_loss_sum / train_steps
        train_acc = float(train_acc_metric.result().numpy())
        train_acc_metric.reset_states()

        val_loss_sum = 0.0
        val_steps = 0

        for x_batch_val, y_batch_val in val_ds:
            logits_val = model(x_batch_val, training=False)
            loss_val = loss_fn(y_batch_val, logits_val)
            if model.losses:
                loss_val += tf.add_n(model.losses)

            val_acc_metric.update_state(y_batch_val, logits_val)

            val_loss_sum += float(loss_val.numpy())
            val_steps += 1

        val_loss = val_loss_sum / val_steps
        val_acc = float(val_acc_metric.result().numpy())
        val_acc_metric.reset_states()

        print(
            f"  train_loss={train_loss:.4f}  "
            f"train_acc={train_acc:.4f}  "
            f"val_loss={val_loss:.4f}  "
            f"val_acc={val_acc:.4f}"
        )

        history.append(
            {
                "epoch": epoch + 1,
                "train_loss": train_loss,
                "train_acc": train_acc,
                "val_loss": val_loss,
                "val_acc": val_acc,
            }
        )

        if val_acc > best_val_acc + 1e-4:
            best_val_acc = val_acc
            best_epoch = epoch + 1
            best_weights = model.get_weights()
            epochs_without_improve = 0
        else:
            epochs_without_improve += 1
            if patience is not None and epochs_without_improve >= patience:
                print(
                    f"Early stopping: sin mejora en val_acc durante {patience} épocas."
                )
                break

    return history, best_weights, best_epoch
